In [4]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [5]:
vm = input()
if vm.lower()=='yes':
    vm=True
else:
    vm=False

yes


In [9]:
if vm:
    path='/mnt/cephfs/ml_data/mc_2021/processed_data/ProcessedTrainReal/'
    data_real = pd.read_csv('{}ProcessedTrain_0.csv.gz'.format(path))
    data_real.append(pd.read_csv('{}ProcessedTrain_100.csv.gz'.format(path)))
    data_real.append(pd.read_csv('{}ProcessedTrain_200.csv.gz'.format(path)))
else:
    data_real = pd.read_csv('processed_data/ProcessedTrainReal/ProcessedTrain.csv.gz')
data_real = data_real[data_real['edepR'] < 17.2]

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/cephfs/ml_data/mc_2021/processed_data/ProcessedTrainReal/ProcessedTrain_100.csv.gz'

In [8]:
data_real

,AccumCharge,nPMTs,R_cc,pho_cc,x_cc,y_cc,z_cc,gamma_z_cc,gamma_y_cc,gamma_x_cc,...,pe_95p,pe_96p,pe_97p,pe_98p,pe_99p,edep,edepX,edepY,edepZ,edepR
0,16795.398556,6698.0,12.025256,8.872896,7.972470,-3.894611,8.116558,0.914759,-0.342320,0.885576,...,8.531833,10.440701,12.698865,15.875377,21.189132,8.309155,10.414397,-5.163487,10.787749,15.858651
1,6658.966649,3645.0,10.909448,10.318122,-8.104657,6.385779,-3.542939,-0.343370,0.721947,-1.109805,...,5.868211,6.806834,7.978919,10.161503,12.634588,3.298830,-12.139917,9.371063,-5.497660,16.291675
2,7296.981869,4395.0,10.514288,9.325840,-1.437017,9.214460,-4.855816,-0.520684,1.819606,-0.137967,...,4.598118,5.096357,5.661463,6.343645,7.667880,3.555040,-1.927146,12.013859,-6.652611,13.867370
3,14586.374299,8350.0,6.747823,4.152723,0.959516,-4.040351,5.318648,1.280762,-0.747589,0.143656,...,4.278572,4.571887,5.073629,5.766348,6.876392,7.941365,1.193331,-5.172269,6.760569,8.595446
5,11599.028010,5631.0,11.202894,3.015708,-1.945551,-2.304198,10.789362,3.577721,-0.210172,-0.176345,...,6.491171,7.242658,8.537913,10.235875,12.480641,6.084913,-2.486934,-3.070554,14.368409,14.901823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499991,17645.792096,9566.0,6.401662,6.399485,-3.613185,5.281884,-0.166932,-0.026085,1.460278,-0.683730,...,4.554131,4.881093,5.325058,6.036111,7.241303,9.661757,-4.187454,6.428227,-0.329893,7.678913
499992,4527.875238,3202.0,9.583840,6.761842,-6.415191,2.137248,6.791722,1.004419,0.228766,-0.901002,...,3.244383,3.497992,3.986275,4.594840,5.805680,2.209429,-8.634691,3.063204,9.308643,13.061085
499993,8836.093129,6091.0,5.786115,5.328440,-5.242109,-0.955284,-2.255405,-0.423277,-0.167397,-2.140185,...,3.278268,3.514569,3.774848,4.445277,5.633961,4.617710,-6.977357,-1.314527,-3.055542,7.729672
499994,5310.483546,4135.0,4.705767,4.704155,-2.976096,3.643066,-0.123157,-0.026181,1.223062,-0.816454,...,2.602201,2.817573,3.239565,3.796524,4.667874,2.772685,-3.901820,5.120572,0.203931,6.440966


In [3]:
N = int(5e5)

X = data_real.iloc[:, :-5]
y = data_real.iloc[:, -5]

X_train = X[:N]
y_train = y[:N]

X_val = X[N:]
y_val = y[N:]

In [5]:
import pickle
xgbreg = pickle.load(open("models/xgb_energy_real.dat", "rb"))
y_predict = xgbreg.predict(X_val)

In [6]:
from sklearn.metrics import mean_squared_error
all_features_rmse = mean_squared_error(y_val, y_predict)**0.5
all_features_rmse

0.08370266641937904

In [7]:
from xgboost import XGBRegressor

opt_features = []
current_rmses = []
current_rmse = 100

features = X_train.columns
while abs(all_features_rmse - current_rmse) > 0.0002:
    rmses = []
    for feature in tqdm(features, "Features loop"):
        xgbreg = XGBRegressor(
            max_depth=9,
            learning_rate=0.08,
            n_estimators=3000,
            random_state=22,
        )
        
        xgbreg.fit(X_train[opt_features+[feature]], y_train, verbose=False,
                   eval_set=[(X_val[opt_features+[feature]], y_val)],
                   early_stopping_rounds=7)
        
        y_predict = xgbreg.predict(X_val[opt_features+[feature]])
        rmse = mean_squared_error(y_val, y_predict)**0.5
        rmses.append(rmse)
    
    best_rmse_ind = np.argmin(rmses)
    current_rmse = rmses[best_rmse_ind]
    current_rmses.append(current_rmse)
    opt_features.append(features[best_rmse_ind])
    features = features.drop(features[best_rmse_ind])

    print(current_rmses)
    print(opt_features)

Features loop:   0%|          | 0/106 [00:00<?, ?it/s]

[0.29581919183216415]
['AccumCharge']


Features loop:   0%|          | 0/105 [00:00<?, ?it/s]

[0.29581919183216415, 0.12584598058842072]
['AccumCharge', 'R_cht']


Features loop:   0%|          | 0/104 [00:00<?, ?it/s]

...

[0.29581919183216415, 0.12584598058842072, 0.09795718313210483, 0.08594291184221113, 0.08480974405042735, 0.08452724565102354, 0.08428262336588634]
['AccumCharge', 'R_cht', 'z_cc', 'pe_std', 'nPMTs', 'pe_mean', 'ht_std']


Features loop:   0%|          | 0/99 [00:00<?, ?it/s]

[0.29581919183216415, 0.12584598058842072, 0.09795718313210483, 0.08594291184221113, 0.08480974405042735, 0.08452724565102354, 0.08428262336588634, 0.08396141732193628]
['AccumCharge', 'R_cht', 'z_cc', 'pe_std', 'nPMTs', 'pe_mean', 'ht_std', 'sin_theta_cc']


Features loop:   0%|          | 0/98 [00:00<?, ?it/s]

[0.29581919183216415, 0.12584598058842072, 0.09795718313210483, 0.08594291184221113, 0.08480974405042735, 0.08452724565102354, 0.08428262336588634, 0.08396141732193628, 0.08375710047382079]
['AccumCharge', 'R_cht', 'z_cc', 'pe_std', 'nPMTs', 'pe_mean', 'ht_std', 'sin_theta_cc', 'phi_cc']


In [8]:
np.savez_compressed('feature_selection/opt_features.npz', a=np.array(opt_features))
np.savez_compressed('feature_selection/current_rmses.npz', a=np.array(current_rmses))